In [ ]:
# default_exp dash

# Dash

> Plotly Dash application to visualize and recommendations

https://plot.ly/dash/

In [ ]:
#hide
from nbdev.export import *

In [ ]:
#export
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

from functools import lru_cache
from pathlib import Path

import pandas as pd
import numpy as np
import requests

from word2vec_recommender.core import *
from word2vec_recommender.recommender import KnnRecommender
from word2vec_recommender.tmdb import load_api_key, TmdbApi, MovieData

Load data frames

In [ ]:
#export
movies_df = pd.read_csv("./data/ml-latest-small/movies.csv")
links_df = pd.read_csv("./data/ml-latest-small/links.csv")

Load Tmdb Api

In [ ]:
#export
movielens_to_tmdb_lookup = { int(movie_id):int(tmdb_id)  for movie_id, tmdb_id in links_df.set_index("movieId")["tmdbId"].to_dict().items() if movie_id > 0 and tmdb_id > 0 }
api = TmdbApi(api_key=load_api_key(), movielens_to_tmdb_lookup=movielens_to_tmdb_lookup)

In [ ]:
#export
with open(Path("./data/out/embeddings.pkl"), "rb") as f:
    embeddings = np.load(f)
with open(Path("./data/out/words_index.pkl"), "rb") as f:
    word_indexes = np.load(f)

movie_id_to_index_lookup = {int(movie_id):idx for idx, movie_id in enumerate(word_indexes)}

knn_recommender = KnnRecommender(
    word_indexes=word_indexes, 
    embeddings=embeddings)

knn_recommender.fit()

In [ ]:
#export
external_stylesheets = []

app = dash.Dash(__name__, 
    external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    html.H1(children='MovieLens Recommender based on Word2Vec'),

    html.Div(children='''
        Dash: A web application framework for Python.
    '''),
    dcc.Dropdown(
        id="movie-id",
        options=movies_df[["title", "movieId"]].apply(lambda r:  {"value": r["movieId"], "label": r["title"]} ,axis=1).tolist(),
    ),
    html.Div(id="selected-movie-id", children="")
])

def movie_card(movie: MovieData, seed: bool = False) -> html.Div:
    description = f"{movie.title} (id={movie.movie_id}, tmdb_id={movie.tmdb_id})"
    return html.Div(children=[
        html.H2(description) if seed else html.H3(description),
        html.Img(src=movie.image_url)
    ])

@app.callback(
    Output(component_id='selected-movie-id', component_property='children'),
    [Input(component_id='movie-id', component_property='value')]
)
def update_output_div(movie_id) -> html.Div:
    if not movie_id:
        return html.Div(children="No movie selected")
    print(movie_id)
    movie = api.fetch_movie_data_by_movielens_id(int(movie_id))
    movie_index = movie_id_to_index_lookup.get(int(movie_id))
    movie_recs = knn_recommender.recommend_by_index(movie_index)
    movies = [ api.fetch_movie_data_by_movielens_id(rec.movie_id) for rec in movie_recs ]
    return html.Div(children=[
        movie_card(movie, seed=True),
        html.Div(children=[
            html.H2("Recommendations"),
            *[movie_card(m) for m in movies]
            ])
        ]) 

In [ ]:
#export
def start_dash(debug: bool = True):
    app.run_server(debug=debug, port=9998)

In [ ]:
#hide   
start_dash(False)

.7/site-packages/dash/dash.py", line 1337, in add_context
    output_value = func(*args, **kwargs)  # %% callback invoked %%
  File "<ipython-input-52-4195976523c6>", line 37, in update_output_div
    movie_recs = knn_recommender.recommend_by_index(movie_index)
  File "/Users/felipe.gasparini/Projects/udacity/ml-engineer/udacity_mle_word2vec_recommender/word2vec_recommender/recommender.py", line 51, in recommend_by_index
    distances_array, indexes_array = self.nn_model.kneighbors([embedding])
  File "/usr/local/Caskroom/miniconda/base/envs/udacity_project/lib/python3.7/site-packages/sklearn/neighbors/base.py", line 402, in kneighbors
    X = check_array(X, accept_sparse='csr')
  File "/usr/local/Caskroom/miniconda/base/envs/udacity_project/lib/python3.7/site-packages/sklearn/utils/validation.py", line 539, in check_array
    % (array.ndim, estimator_name))
ValueError: Found array with dim 4. Estimator expected <= 2.
Exception on /_dash-update-component [POST]
Traceback (most recent c